<a href="https://colab.research.google.com/github/Bannore-Aaditya/Bannore-Aaditya/blob/main/PGMAssignment2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pgmpy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 2.1 MB/s 


In [2]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

Setting model

In [3]:
difficulty_cpd = TabularCPD(variable='D',
                       variable_card=2,
                       values=[[.6], [.4]])

intelligence_cpd = TabularCPD(variable='I',
                       variable_card=2,
                       values=[[.7], [.3]])

sat_cpd = TabularCPD(variable='S',
                     variable_card=2,
                     values=[[.95, 0.2],
                             [.05, 0.8]],
                     evidence=['I'],
                     evidence_card=[2])

# grade
grade_cpd = TabularCPD(variable='G',
                         variable_card=3,
                         values=[[.3, .05, .9, .5 ],
                        [.4, .25, .08, .3],
                        [.3, .7, .02, .2]],
                         evidence=['I', 'D'],
                         evidence_card=[2, 2])

letter_cpd = TabularCPD(variable='L',
                     variable_card=2,
                     values=[[.1, 0.4, .99],
                             [.9, 0.6, .01]],
                     evidence=['G'],
                     evidence_card=[3])

Building the Student Model

In [4]:
student_model = BayesianModel([('D', 'G'),('I', 'G'), ('I', 'S'), ('G', 'L')])

/usr/local/lib/python3.7/dist-packages/pgmpy/models/BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  FutureWarning,


In [5]:
student_model.add_cpds(difficulty_cpd, intelligence_cpd, sat_cpd, grade_cpd, letter_cpd)

In [6]:
print("CPDS")
print(student_model.get_cpds())
#student_model.get_leaves()
print("Independencies")
print(student_model.get_independencies())

CPDS
[<TabularCPD representing P(D:2) at 0x7fdee1be8610>, <TabularCPD representing P(I:2) at 0x7fdee1be85d0>, <TabularCPD representing P(S:2 | I:2) at 0x7fdee1be8650>, <TabularCPD representing P(G:3 | I:2, D:2) at 0x7fdee1be8690>, <TabularCPD representing P(L:2 | G:3) at 0x7fdee1be8710>]
Independencies
(I ⟂ D)
(I ⟂ L | G)
(I ⟂ D | S)
(I ⟂ L | G, D)
(I ⟂ L | G, S)
(I ⟂ L | G, D, S)
(G ⟂ S | I)
(G ⟂ S | I, D)
(G ⟂ S | I, L)
(G ⟂ S | I, D, L)
(D ⟂ I, S)
(D ⟂ S | I)
(D ⟂ L | G)
(D ⟂ I | S)
(D ⟂ S, L | I, G)
(D ⟂ S | I, L)
(D ⟂ L | G, S)
(D ⟂ L | I, G, S)
(D ⟂ S | I, G, L)
(S ⟂ D)
(S ⟂ G, D, L | I)
(S ⟂ L | G)
(S ⟂ D, L | I, G)
(S ⟂ G, L | I, D)
(S ⟂ G, D | I, L)
(S ⟂ L | G, D)
(S ⟂ L | I, G, D)
(S ⟂ D | I, G, L)
(S ⟂ G | I, D, L)
(L ⟂ S | I)
(L ⟂ I, S, D | G)
(L ⟂ S, D | I, G)
(L ⟂ S | I, D)
(L ⟂ I, S | G, D)
(L ⟂ I, D | G, S)
(L ⟂ S | I, G, D)
(L ⟂ D | I, G, S)
(L ⟂ I | G, D, S)


In [8]:
from pgmpy.inference.base import Inference
from pgmpy.factors import factor_product

import itertools


class SimpleInference(Inference):
    def query(self, var, evidence):
        # self.factors is a dict of the form of {node: [factors_involving_node]}
        factors_list = set(itertools.chain(*self.factors.values()))
        product = factor_product(*factors_list)
        reduced_prod = product.reduce(evidence, inplace=False)
        reduced_prod.normalize()
        var_to_marg = set(self.model.nodes()) - set(var) - set([state[0] for state in evidence])
        marg_prod = reduced_prod.marginalize(var_to_marg, inplace=False)
        return marg_prod


infer = SimpleInference(student_model)
l1 = infer.query(var=['L'], evidence=[('I', 1), ('D', 0)])
print(l1)

AttributeError: ignored

In [9]:
from pgmpy.inference import VariableElimination

olympic_infer = VariableElimination(student_model)

Questions

In [10]:
prob_offer = olympic_infer.query(variables = ['L'])
print(prob_offer)

+------+----------+
| L    |   phi(L) |
+======+==========+
| L(0) |   0.4977 |
+------+----------+
| L(1) |   0.5023 |
+------+----------+


/usr/local/lib/python3.7/dist-packages/pgmpy/models/BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  FutureWarning,


In [11]:
prob_offer_bad_genes = olympic_infer.query(
                                        variables = ['L'], 
                                        evidence = {'D':1})
print(prob_offer_bad_genes)

+------+----------+
| L    |   phi(L) |
+======+==========+
| L(0) |   0.6690 |
+------+----------+
| L(1) |   0.3310 |
+------+----------+


In [14]:
prob_offer_good_genes_did_practice = olympic_infer.query(
                                        variables = ['L'], 
                                        evidence = {'D':0, 'I':0})
print(prob_offer_good_genes_did_practice)

+------+----------+
| L    |   phi(L) |
+======+==========+
| L(0) |   0.4870 |
+------+----------+
| L(1) |   0.5130 |
+------+----------+


In [16]:
prob_good_genes_if_amazing_olympic_trials = olympic_infer.query(
                                        variables = ['G'], 
                                        evidence = {'L':0})
print(prob_good_genes_if_amazing_olympic_trials)

+------+----------+
| G    |   phi(G) |
+======+==========+
| G(0) |   0.0727 |
+------+----------+
| G(1) |   0.2318 |
+------+----------+
| G(2) |   0.6955 |
+------+----------+
